# E2E Networks LLM Integration Tutorial

## 📚 Complete Guide to Using E2E Networks LLM with Python

This notebook demonstrates step-by-step how to:
1. Initialize and use E2E Networks LLM with LangChain
2. Make direct API calls to E2E endpoints
3. Handle responses and errors
4. Compare different approaches

---

## 🔧 Step 1: Environment Setup and Imports

First, let's import all necessary libraries and set up our environment.

In [ ]:
# Import required libraries
import os
import requests
import json
from typing import Optional, List, Any
from dotenv import load_dotenv

# LangChain imports
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

# For OpenAI-compatible approach
import openai

print("✅ All libraries imported successfully!")

## 🔑 Step 2: Load Environment Variables

Load our E2E Networks credentials from the .env file.

In [ ]:
# Load environment variables
load_dotenv()

# Get E2E Networks credentials
E2E_ENDPOINT_URL = os.getenv("E2E_ENDPOINT_URL")
E2E_API_KEY = os.getenv("E2E_API_KEY")

print(f"Endpoint URL: {E2E_ENDPOINT_URL}")
print(f"API Key: {'*' * 20}...{E2E_API_KEY[-10:] if E2E_API_KEY else 'Not found'}")

## 🎯 Method 1: Custom LangChain LLM Wrapper

Let's create a custom LangChain LLM wrapper for E2E Networks.

In [ ]:
class E2ENetworksLLM(LLM):
    """Custom LLM wrapper for E2E Networks endpoint"""
    
    endpoint_url: str = ""
    api_key: str = ""
    model_name: str = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    temperature: float = 0.7
    max_tokens: int = 1000
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.endpoint_url = E2E_ENDPOINT_URL
        self.api_key = E2E_API_KEY
    
    @property
    def _llm_type(self) -> str:
        return "e2e_networks_llm"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Call the E2E LLM endpoint using OpenAI-compatible format"""
        
        print(f"🚀 Making API call to: {self.endpoint_url}")
        print(f"📝 Prompt: {prompt[:100]}...")
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        # Use OpenAI-compatible chat completions format
        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }
        
        if stop:
            payload["stop"] = stop
        
        try:
            # Ensure endpoint ends with chat/completions
            endpoint = self.endpoint_url
            if not endpoint.endswith("chat/completions"):
                if endpoint.endswith("/"):
                    endpoint += "chat/completions"
                else:
                    endpoint += "/chat/completions"
            
            print(f"🌐 Full endpoint: {endpoint}")
            
            response = requests.post(
                endpoint,
                headers=headers,
                json=payload,
                timeout=30
            )
            response.raise_for_status()
            
            result = response.json()
            print(f"✅ Response received: {len(str(result))} characters")
            
            # Handle OpenAI-compatible response format
            if "choices" in result and len(result["choices"]) > 0:
                choice = result["choices"][0]
                if "message" in choice:
                    return choice["message"].get("content", "")
                elif "text" in choice:
                    return choice["text"]
            
            return "No valid response received"
                
        except requests.exceptions.RequestException as e:
            error_msg = f"❌ Error calling E2E LLM: {str(e)}"
            print(error_msg)
            return error_msg
        except Exception as e:
            error_msg = f"❌ Unexpected error: {str(e)}"
            print(error_msg)
            return error_msg

print("✅ E2ENetworksLLM class defined successfully!")

## 🧪 Step 3: Initialize and Test LangChain LLM

Now let's create an instance of our custom LLM and test it.

In [ ]:
# Initialize the LLM
llm = E2ENetworksLLM(
    temperature=0.7,
    max_tokens=500
)

print("🎯 LLM initialized with:")
print(f"   Model: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")
print(f"   Max Tokens: {llm.max_tokens}")
print(f"   LLM Type: {llm._llm_type}")

In [ ]:
# Test the LLM with a simple prompt
test_prompt = "What is Python programming language? Give a brief explanation."

print("🧪 Testing LLM with prompt:")
print(f"'{test_prompt}'")
print("\n" + "="*50)

response = llm._call(test_prompt)

print("\n" + "="*50)
print("🤖 LLM Response:")
print(response)

## 🔄 Method 2: Direct OpenAI-Compatible API Call

Let's also demonstrate how to make direct API calls using the OpenAI library.

In [ ]:
# Configure OpenAI client for E2E Networks
openai.api_key = E2E_API_KEY
openai.base_url = E2E_ENDPOINT_URL

print("🔧 OpenAI client configured for E2E Networks")
print(f"Base URL: {openai.base_url}")
print(f"API Key: {'*' * 20}...{E2E_API_KEY[-10:]}")

In [ ]:
# Make a direct API call using OpenAI format
def call_e2e_llm_direct(prompt, temperature=0.7, max_tokens=500):
    """Direct API call to E2E Networks LLM"""
    
    print(f"🚀 Making direct API call...")
    print(f"📝 Prompt: {prompt[:100]}...")
    
    try:
        completion = openai.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        response_content = completion.choices[0].message.content
        print(f"✅ Response received: {len(response_content)} characters")
        
        return response_content
        
    except Exception as e:
        error_msg = f"❌ Error in direct API call: {str(e)}"
        print(error_msg)
        return error_msg

# Test the direct API call
direct_prompt = "Write a Python function to calculate factorial of a number."

print("🧪 Testing direct API call with prompt:")
print(f"'{direct_prompt}'")
print("\n" + "="*50)

direct_response = call_e2e_llm_direct(direct_prompt)

print("\n" + "="*50)
print("🤖 Direct API Response:")
print(direct_response)

## 📊 Step 4: Compare Different Approaches

Let's compare the LangChain wrapper vs direct API calls.

In [ ]:
import time

# Test prompt for comparison
comparison_prompt = "Explain the difference between machine learning and deep learning in simple terms."

print("🔄 Comparing LangChain vs Direct API approaches...")
print(f"Test prompt: '{comparison_prompt}'")
print("\n" + "="*70)

# Method 1: LangChain
print("\n🔗 Method 1: LangChain Wrapper")
start_time = time.time()
langchain_response = llm._call(comparison_prompt)
langchain_time = time.time() - start_time

print(f"⏱️ Time taken: {langchain_time:.2f} seconds")
print(f"📝 Response length: {len(langchain_response)} characters")
print(f"🤖 Response: {langchain_response[:200]}...")

print("\n" + "-"*70)

# Method 2: Direct API
print("\n🌐 Method 2: Direct OpenAI API")
start_time = time.time()
direct_response = call_e2e_llm_direct(comparison_prompt)
direct_time = time.time() - start_time

print(f"⏱️ Time taken: {direct_time:.2f} seconds")
print(f"📝 Response length: {len(direct_response)} characters")
print(f"🤖 Response: {direct_response[:200]}...")

print("\n" + "="*70)
print("📈 Performance Comparison:")
print(f"   LangChain: {langchain_time:.2f}s")
print(f"   Direct API: {direct_time:.2f}s")
print(f"   Difference: {abs(langchain_time - direct_time):.2f}s")

## 🛠️ Step 5: Advanced Examples

Let's explore more advanced usage patterns.

In [ ]:
# Example 1: Code generation
code_prompt = """
Create a Python class for a simple bank account with the following methods:
1. __init__(self, account_number, initial_balance)
2. deposit(self, amount)
3. withdraw(self, amount)
4. get_balance(self)

Include proper error handling and documentation.
"""

print("💻 Code Generation Example:")
print("=" * 50)

code_response = llm._call(code_prompt)
print(code_response)

In [ ]:
# Example 2: Creative writing
creative_prompt = """
Write a short story (3-4 paragraphs) about a programmer who discovers 
that their AI assistant has developed consciousness. Make it engaging and thought-provoking.
"""

print("✍️ Creative Writing Example:")
print("=" * 50)

creative_response = call_e2e_llm_direct(creative_prompt, temperature=0.9, max_tokens=800)
print(creative_response)

In [ ]:
# Example 3: Technical explanation
technical_prompt = """
Explain how neural networks work, including:
1. Basic structure (neurons, layers)
2. Forward propagation
3. Backpropagation
4. Training process

Use simple language suitable for beginners.
"""

print("🧠 Technical Explanation Example:")
print("=" * 50)

technical_response = llm._call(technical_prompt)
print(technical_response)

## 🔍 Step 6: Error Handling and Best Practices

Let's demonstrate proper error handling and best practices.

In [ ]:
def robust_llm_call(prompt, max_retries=3, timeout=30):
    """Robust LLM call with retry logic and error handling"""
    
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries}")
            
            # Use the LangChain wrapper
            response = llm._call(prompt)
            
            # Validate response
            if response and not response.startswith("Error"):
                print(f"✅ Success on attempt {attempt + 1}")
                return response
            else:
                print(f"⚠️ Invalid response on attempt {attempt + 1}")
                
        except Exception as e:
            print(f"❌ Error on attempt {attempt + 1}: {str(e)}")
            
        if attempt < max_retries - 1:
            print(f"⏳ Waiting before retry...")
            time.sleep(2)  # Wait 2 seconds before retry
    
    return "Failed to get response after all retries"

# Test robust calling
test_prompt = "What are the key principles of good software design?"

print("🛡️ Testing robust LLM calling:")
print("=" * 50)

robust_response = robust_llm_call(test_prompt)
print("\n📋 Final Response:")
print(robust_response)

## 📋 Step 7: Summary and Key Takeaways

Let's summarize what we've learned and provide key takeaways.

In [ ]:
print("📚 E2E Networks LLM Integration - Summary")
print("=" * 60)
print()
print("🎯 What we covered:")
print("   1. ✅ Custom LangChain LLM wrapper creation")
print("   2. ✅ Direct OpenAI-compatible API calls")
print("   3. ✅ Performance comparison between methods")
print("   4. ✅ Advanced usage examples (code, creative, technical)")
print("   5. ✅ Error handling and best practices")
print()
print("🔑 Key Takeaways:")
print("   • E2E Networks uses OpenAI-compatible API format")
print("   • LangChain wrapper provides better integration with LangChain ecosystem")
print("   • Direct API calls offer more control and transparency")
print("   • Proper error handling is crucial for production use")
print("   • Temperature and max_tokens significantly affect output")
print()
print("🚀 Next Steps:")
print("   • Integrate with your applications")
print("   • Experiment with different prompting techniques")
print("   • Build more sophisticated LLM-powered features")
print("   • Monitor usage and optimize for your use case")
print()
print("🎉 You're now ready to use E2E Networks LLM in your projects!")

---

## 📚 Additional Resources

- **E2E Networks Documentation**: Check your E2E Networks dashboard for API documentation
- **LangChain Documentation**: https://python.langchain.com/
- **OpenAI API Reference**: https://platform.openai.com/docs/api-reference

## 🤝 Support

If you encounter any issues:
1. Check your API credentials and endpoint URL
2. Verify network connectivity
3. Review error messages carefully
4. Contact E2E Networks support for API-related issues

---

**Happy coding with E2E Networks LLM! 🚀**